In [2]:
# import keras
# from keras.datasets import cifar10
# from keras.models import Model, Sequential
# from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
#from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# from keras.layers import Concatenate
# from keras.optimizers import Adam
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import he_uniform
import os.path

import numpy as np
import sklearn.metrics as metrics

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K

In [3]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf

In [4]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10) 

170500096/170498071 [==============================] - 11s 0us/step


In [5]:
X_train.shape

(50000, 32, 32, 3)

In [6]:
X_test.shape

(10000, 32, 32, 3)

In [7]:
data_format = K.image_data_format()

In [8]:
K.common.image_dim_ordering()

'tf'

In [9]:
data_format

'channels_last'

In [10]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [11]:
generator = ImageDataGenerator(rotation_range=15,
                               width_shift_range=5./32,
                               height_shift_range=5./32,
                               horizontal_flip=True)

generator.fit(X_train, seed=0)

In [12]:
batch_size = 128
nb_epoch = 300

In [13]:
# Dense Block
def denseblock(input, num_filter = 42):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization(epsilon=1.1e-5)(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), kernel_initializer ="he_normal", use_bias=False ,padding='same')(relu)
        #Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 42):
    global compression
    BatchNorm = layers.BatchNormalization(epsilon=1.1e-5)(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), kernel_initializer ="he_normal", use_bias=False ,padding='same', kernel_regularizer=l2(1e-4))(relu)
    #Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    features_total = int(flat.get_shape()[-1])
    flat = tf.reshape(flat, [-1, features_total])
    #print(flat.shape)
    
    #https://towardsdatascience.com/neural-network-with-tensorflow-how-to-stop-training-using-callback-5c8d575c18a9
    initializer = tf.keras.initializers.GlorotNormal()
    W = initializer(shape=(features_total, 10))
    
    initializer2 = tf.keras.initializers.Constant(0.0)
    bias = initializer2(shape=(10))
    out = tf.matmul(flat, W) + bias
    
    output = tf.math.exp(out) # exponentiate vector of raw predictions
    #print(output)
    out = tf.math.reduce_sum(output)
    return output/out

    #output = layers.Dense(10, activation='softmax')(flat)
    #return output

#output layer
def output_layer1(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(10, activation='softmax')(flat)
    return output

In [14]:
num_filter = 42
l = 10
compression = 0.5
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), activation="relu", strides=(2,2), kernel_initializer ="he_uniform", use_bias=False ,padding='same', kernel_regularizer=l2(1e-4))(input)
First_Conv2D = layers.BatchNormalization(axis=-1, epsilon=1.1e-5)(First_Conv2D)
#First_Conv2D = layers.MaxPooling2D((3, 3), strides=2, padding='same',data_format='channels_last')(First_Conv2D)
#First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter)
First_Transition = transition(First_Block, num_filter)

Second_Block = denseblock(First_Transition, num_filter)
Second_Transition = transition(Second_Block, num_filter)

Third_Block = denseblock(Second_Transition, num_filter)
Third_Transition = transition(Third_Block, num_filter)

Last_Block = denseblock(Third_Transition,  num_filter)
output = output_layer(Last_Block)

#outputs = [model(model_inputs) for model in models]

In [15]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 16, 16, 42)   1134        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 16, 16, 42)   168         conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 16, 16, 42)   168         batch_normalization[0][0]        
______________________________________________________________________________________________

In [20]:
#https://stackoverflow.com/questions/59889940/how-to-write-custom-callback-for-saving-the-model-at-every-epoch-if-validation-a
#https://stackoverflow.com/questions/47079111/create-keras-callback-to-save-model-predictions-and-targets-for-each-batch-durin

class CustomLearningRateScheduler(tf.keras.callbacks.Callback):
    
    def __init__(self, schedule):
        super(CustomLearningRateScheduler, self).__init__()
        self.schedule = schedule
        
    def on_epoch_begin(self, epoch, logs={}):

        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        # Get the current learning rate from model's optimizer.
        #lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        # Call schedule function to get the scheduled learning rate.
        scheduled_lr = self.schedule(epoch)
        # Set the value back to the optimizer before this epoch starts
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        print("\nEpoch %05d: Learning rate is %6.4f." % (epoch, scheduled_lr))

    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('val_accuracy') > 0.9):   
          print("\nReached %2.2f%% accuracy, so stopping training!!" %(0.9*100))   
          self.model.stop_training = True
        

In [17]:
def lr_schedule(epoch):
    """Helper function to retrieve the scheduled learning rate based on epoch."""
    lr = 0.1
    if epoch == 150 or epoch >=290:
        lr = 0.01
    
    return lr

In [21]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(momentum = 0.9, learning_rate=0.1),
              metrics=['accuracy'])
'''
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])'''

"\nmodel.compile(loss='categorical_crossentropy',\n              optimizer=Adam(),\n              metrics=['accuracy'])"

In [22]:
model.fit_generator(generator.flow(X_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(X_train) // batch_size, epochs=nb_epoch,
                    callbacks=[CustomLearningRateScheduler(lr_schedule)],
                    validation_data=(X_test, y_test),
                    validation_steps=X_test.shape[0] // batch_size, verbose=1)



Epoch 00000: Learning rate is 0.1000.
Epoch 1/300
390/390 [==============================] - 28s 71ms/step - loss: 1.3153 - accuracy: 0.5326 - val_loss: 1.5850 - val_accuracy: 0.4804

Epoch 00001: Learning rate is 0.1000.
Epoch 2/300
390/390 [==============================] - 27s 69ms/step - loss: 1.1420 - accuracy: 0.6031 - val_loss: 1.9643 - val_accuracy: 0.4390

Epoch 00002: Learning rate is 0.1000.
Epoch 3/300
390/390 [==============================] - 27s 69ms/step - loss: 1.0259 - accuracy: 0.6479 - val_loss: 1.2476 - val_accuracy: 0.5942

Epoch 00003: Learning rate is 0.1000.
Epoch 4/300
390/390 [==============================] - 27s 70ms/step - loss: 0.9392 - accuracy: 0.6815 - val_loss: 1.0799 - val_accuracy: 0.6553

Epoch 00004: Learning rate is 0.1000.
Epoch 5/300
390/390 [==============================] - 27s 70ms/step - loss: 0.8690 - accuracy: 0.7078 - val_loss: 1.0564 - val_accuracy: 0.6574

Epoch 00005: Learning rate is 0.1000.
Epoch 6/300
390/390 [====================

In [23]:
score = model.evaluate(X_test, y_test, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Test score: 0.37801840901374817
Test accuracy: 0.9045000076293945


In [24]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk
